In [1]:
import pandas as pd
import sys
import spotipy
import spotipy.util as util
import requests
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.io.json import json_normalize
import time

In [35]:
username='jkyung'

my_client_id='my_client_id'
my_client_secret='my_client_secret'
 
scope = 'user-library-read playlist-modify-public playlist-modify-private'

df = pd.read_csv("billboard_hits.csv", index_col = 0)
df.head()

,Year,Songs,Artists
0,2014,Happy,Pharrell Williams
1,2014,Dark Horse,Katy Perry featuring Juicy J
2,2014,All of Me,John Legend
3,2014,Fancy,Iggy Azalea featuring Charli XCX
4,2014,Counting Stars,OneRepublic


In [ ]:
# client_credentials_manager = SpotifyClientCredentials(username, scope,client_id= my_client_id,client_secret= my_client_secret)
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [41]:
token = util.prompt_for_user_token(username, scope,client_id=my_client_id,client_secret=my_client_secret,redirect_uri='http://localhost:8888/')
myAuth="Bearer " + token

sp = spotipy.Spotify(auth=token)

In [42]:
def get_song_track_id(row):
    
    track = row[1]
    track = track.split('(')[0]
    
    artist = row[2]
    if 'featuring' in artist:
        artist = artist.split('featuring')[0]
    elif ' and ' in artist:
        artist = artist.split(' and ')[0]

    query = 'track:' + track + ' artist:' + artist
    track_id = sp.search(q=query , type='track')
    return track_id['tracks']['items'][0]['uri']

In [43]:
df['Track_ID'] = df.apply(get_song_track_id, axis=1)

In [44]:
df.head()

,Year,Songs,Artists,Track_ID
0,2014,Happy,Pharrell Williams,spotify:track:6NPVjNh8Jhru9xOmyQigds
1,2014,Dark Horse,Katy Perry featuring Juicy J,spotify:track:5jrdCoLpJSvHHorevXBATy
2,2014,All of Me,John Legend,spotify:track:3U4isOIWM3VvDubwSI3y7a
3,2014,Fancy,Iggy Azalea featuring Charli XCX,spotify:track:3oiMJQAWVaxSubJ7b2VUtX
4,2014,Counting Stars,OneRepublic,spotify:track:6sy3LkhNFjJWlaeSMNwQ62


In [45]:
features = []

for i,tid in enumerate(df.Track_ID):
    if i%300 == 0:
        time.sleep(10)
    feature = sp.audio_features(tid)
    features.append(feature[0])

In [46]:
features
len(features)

400

In [47]:
features_df = json_normalize(features)
features_df.head()

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.28600,https://api.spotify.com/v1/audio-analysis/6NPV...,0.652,233305,0.757,6NPVjNh8Jhru9xOmyQigds,0.0,1,0.0886,-6.819,1,0.1530,159.911,4,https://api.spotify.com/v1/tracks/6NPVjNh8Jhru...,audio_features,spotify:track:6NPVjNh8Jhru9xOmyQigds,0.962
1,0.00314,https://api.spotify.com/v1/audio-analysis/5jrd...,0.645,215672,0.585,5jrdCoLpJSvHHorevXBATy,0.0,6,0.1650,-6.122,1,0.0513,131.931,4,https://api.spotify.com/v1/tracks/5jrdCoLpJSvH...,audio_features,spotify:track:5jrdCoLpJSvHHorevXBATy,0.353
2,0.92200,https://api.spotify.com/v1/audio-analysis/3U4i...,0.422,269560,0.264,3U4isOIWM3VvDubwSI3y7a,0.0,8,0.1320,-7.064,1,0.0322,119.930,4,https://api.spotify.com/v1/tracks/3U4isOIWM3Vv...,audio_features,spotify:track:3U4isOIWM3VvDubwSI3y7a,0.331
3,0.10200,https://api.spotify.com/v1/audio-analysis/3oiM...,0.911,199938,0.707,3oiMJQAWVaxSubJ7b2VUtX,0.0,8,0.0490,-4.136,1,0.0696,94.964,4,https://api.spotify.com/v1/tracks/3oiMJQAWVaxS...,audio_features,spotify:track:3oiMJQAWVaxSubJ7b2VUtX,0.375
4,0.06490,https://api.spotify.com/v1/audio-analysis/6sy3...,0.663,257840,0.714,6sy3LkhNFjJWlaeSMNwQ62,0.0,1,0.1160,-4.944,0,0.0380,121.990,4,https://api.spotify.com/v1/tracks/6sy3LkhNFjJW...,audio_features,spotify:track:6sy3LkhNFjJWlaeSMNwQ62,0.468


In [48]:
# combine dataframes
df_result = pd.concat([df, features_df], axis=1, join_axes=[df.index])

In [49]:
df_result.tail()

,Year,Songs,Artists,Track_ID,acousticness,analysis_url,danceability,duration_ms,energy,id,...,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
395,2017,Havana,Camila Cabello featuring Young Thug,spotify:track:1rfofaqEpACxVEHIZBJe6W,0.18400,https://api.spotify.com/v1/audio-analysis/1rfo...,0.765,217307,0.523,1rfofaqEpACxVEHIZBJe6W,...,0.1320,-4.333,1,0.0300,104.988,4,https://api.spotify.com/v1/tracks/1rfofaqEpACx...,audio_features,spotify:track:1rfofaqEpACxVEHIZBJe6W,0.394
396,2017,What Lovers Do,Maroon 5 featuring SZA,spotify:track:2j5hsQvApottzvTn4pFJWF,0.07880,https://api.spotify.com/v1/audio-analysis/2j5h...,0.799,199849,0.597,2j5hsQvApottzvTn4pFJWF,...,0.1000,-5.131,0,0.0611,110.001,4,https://api.spotify.com/v1/tracks/2j5hsQvApott...,audio_features,spotify:track:2j5hsQvApottzvTn4pFJWF,0.419
397,2017,Do Re Mi,Blackbear,spotify:track:3Q3myFA7q4Op95DOpHplaY,0.00419,https://api.spotify.com/v1/audio-analysis/3Q3m...,0.742,212027,0.599,3Q3myFA7q4Op95DOpHplaY,...,0.1260,-6.256,1,0.0465,110.977,3,https://api.spotify.com/v1/tracks/3Q3myFA7q4Op...,audio_features,spotify:track:3Q3myFA7q4Op95DOpHplaY,0.154
398,2017,Look at Me!,XXXTentacion,spotify:track:7floNISpH8VF4z4459Qo18,0.25900,https://api.spotify.com/v1/audio-analysis/7flo...,0.763,126346,0.726,7floNISpH8VF4z4459Qo18,...,0.0976,-6.405,1,0.2820,139.059,4,https://api.spotify.com/v1/tracks/7floNISpH8VF...,audio_features,spotify:track:7floNISpH8VF4z4459Qo18,0.349
399,2017,The Fighter,Keith Urban featuring Carrie Underwood,spotify:track:5OUSPcqhYTOzpbXzoEvKim,0.03040,https://api.spotify.com/v1/audio-analysis/5OUS...,0.681,184040,0.845,5OUSPcqhYTOzpbXzoEvKim,...,0.1970,-5.147,0,0.0525,132.023,4,https://api.spotify.com/v1/tracks/5OUSPcqhYTOz...,audio_features,spotify:track:5OUSPcqhYTOzpbXzoEvKim,0.761


In [50]:
# filter only useful columns
df_result.drop(['analysis_url', 'id', 'track_href','uri','type'], inplace = True, axis=1)

In [51]:
df_result.head()

,Year,Songs,Artists,Track_ID,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,2014,Happy,Pharrell Williams,spotify:track:6NPVjNh8Jhru9xOmyQigds,0.28600,0.652,233305,0.757,0.0,1,0.0886,-6.819,1,0.1530,159.911,4,0.962
1,2014,Dark Horse,Katy Perry featuring Juicy J,spotify:track:5jrdCoLpJSvHHorevXBATy,0.00314,0.645,215672,0.585,0.0,6,0.1650,-6.122,1,0.0513,131.931,4,0.353
2,2014,All of Me,John Legend,spotify:track:3U4isOIWM3VvDubwSI3y7a,0.92200,0.422,269560,0.264,0.0,8,0.1320,-7.064,1,0.0322,119.930,4,0.331
3,2014,Fancy,Iggy Azalea featuring Charli XCX,spotify:track:3oiMJQAWVaxSubJ7b2VUtX,0.10200,0.911,199938,0.707,0.0,8,0.0490,-4.136,1,0.0696,94.964,4,0.375
4,2014,Counting Stars,OneRepublic,spotify:track:6sy3LkhNFjJWlaeSMNwQ62,0.06490,0.663,257840,0.714,0.0,1,0.1160,-4.944,0,0.0380,121.990,4,0.468


In [52]:
df_result.to_csv('billboard_hits_with_features.csv')